# Topic Modeling

In [1]:
import pandas as pd
import re

# Reading the data
sb_data = pd.read_csv("spongebob_data.csv")
sb_transcripts = sb_data["transcript"]

# Removes the speaker
def remove_speakers(text):
    output = []
    lines = text.split("\n")
    for line in lines:
        segments = line.split(": ")
        if len(segments) > 1:
            new_text = "".join(segments[1:]) 
        else:
            new_text = "".join(segments)
        output.append(new_text)
    return " ".join(output)

# Removes the speaker, then removes the queues in the brackets
sb_transcripts = sb_transcripts.apply(lambda x: re.sub("[\[].*?[\]]", "", remove_speakers(x)))

sb_transcripts.head()

0     Ah, the sea... so fascinating. So wonderful. ...
1     Ahh, what a wonderful day. The sun is out, th...
2     Ah, ze early morning in Bikini Bottom, when t...
3    Wow. Squidward, this is the best soufflé you h...
4     Hurry up with those chairs, SpongeBob. It's a...
Name: transcript, dtype: object

Formatting and Tokenization

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string

# Run the below line of code to download the list of stopwords:
# nltk.download("stopwords")

# Removes punctuation and turns the word into lowercase
def remove_punc_and_lowercase(text):
    return "".join(s for s in text if not s in string.punctuation).lower()


# Removes punctuation and stop words, puts the text into lowercase, and performs tokenization
def format_and_tokenize(text):
    text = remove_punc_and_lowercase(text)
    stop_words = [remove_punc_and_lowercase(word) for word in stopwords.words("english")]
    no_stop_words = [x for x in word_tokenize(text) if not x in stop_words]
    return no_stop_words

sb_transcripts = sb_transcripts.apply(format_and_tokenize)

sb_transcripts.head()

0    [ah, sea, fascinating, wonderful, see, bikini,...
1    [ahh, wonderful, day, sun, water, shimmering, ...
2    [ah, ze, early, morning, bikini, bottom, jelly...
3    [wow, squidward, best, soufflé, ever, created,...
4    [hurry, chairs, spongebob, closing, id, like, ...
Name: transcript, dtype: object

Lemmatization and POS Filtering

In [3]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# Returns true only if the part of speech is a noun or adjective or verb
def noun_or_adjective_or_verb(pos):
    return pos[:2] in ["NN", "JJ", "VB"]

# Lemmatizes the list and filters to get only nouns and adjectives
def lem_and_filter(text):
    pos_tags = pos_tag(text)
    return [lemmatizer.lemmatize(word) for (word, pos) in pos_tags if noun_or_adjective_or_verb(pos)]

sb_transcripts = sb_transcripts.apply(lem_and_filter)

sb_transcripts.head()

0    [sea, fascinating, wonderful, see, bikini, bot...
1    [ahh, wonderful, day, sun, water, shimmering, ...
2    [ah, early, morning, bikini, bottom, jellyfish...
3    [best, soufflé, created, congratulation, chef,...
4    [hurry, chair, spongebob, closing, id, go, hom...
Name: transcript, dtype: object

Model

In [38]:
import gensim.corpora as corpora
from gensim.models import CoherenceModel

# Adding all of the words into one list
all_words = sb_transcripts.to_list()

id2word = corpora.Dictionary(all_words)

corpus = [id2word.doc2bow(words) for words in all_words]
    
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word = id2word, num_topics = 4, 
                                            random_state = 100, update_every = 1, chunksize = 100,
                                            passes = 10, alpha = "auto")

Model Visualization

In [39]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

C:\Users\ottob\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.060324  0.034438       1        1  37.910960
1      0.016403  0.067503       2        1  32.964206
3      0.081261 -0.017561       3        1  15.374038
0     -0.037340 -0.084380       4        1  13.750796, topic_info=         Term        Freq        Total Category  logprob  loglift
176      meow  726.000000   726.000000  Default  30.0000  30.0000
92       gary  787.000000   787.000000  Default  29.0000  29.0000
677  plankton  646.000000   646.000000  Default  28.0000  28.0000
615   formula  380.000000   380.000000  Default  27.0000  27.0000
169       man  741.000000   741.000000  Default  26.0000  26.0000
..        ...         ...          ...      ...      ...      ...
552      yeah  149.405201  1115.995887   Topic4  -5.4746  -0.0268
237       see  164.884108  1517.603188   Topic4  -5.3760  -0.2356
473     right  143.850319  1019.674996   Topic4  -5.5125   0.0256
134       ill  154.779281  1322.149289   Topic4  -5.4392  -0.1609
205   patrick  172.340276  2207.478860   Topic4  -5.3318  -0.5661

[364 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
8637       3  0.174030     amoeba
8637       4  0.835343     amoeba
563        1  0.930107     artist
563        2  0.067644     artist
9638       3  0.903042  atlantean
...      ...       ...        ...
552        3  0.143370       yeah
552        4  0.133513       yeah
335        1  0.917048          ♪
335        2  0.080443          ♪
12664      1  0.926019    ♪rodeo♪

[620 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 1])